# Final Step! Merge all the data and do analysis 

In [13]:
import geopandas as gpd
import pandas as pd


In [14]:
gdf_trees = gpd.read_file("data/berlin_strassenbaeume.geojson")
gdf_trees.columns


Index(['id', 'gisid', 'pitid', 'standortnr', 'kennzeich', 'namenr',
       'art_dtsch', 'art_bot', 'gattung_deutsch', 'gattung', 'art_gruppe',
       'strname', 'hausnr', 'zusatz', 'pflanzjahr', 'standalter', 'kronedurch',
       'stammumfg', 'baumhoehe', 'eigentuemer', 'bezirk', 'geometry'],
      dtype='object')

In [15]:
df_ndvi = pd.read_csv("data/timeseries/ndvi_CWP_2022.csv")
df_water = pd.read_csv("data/timeseries/watering_2022.csv")  
df_weather = pd.read_csv("data/timeseries/weather_features.csv")  

In [16]:
df_ndvi["tree_id"] = df_ndvi["tree_id"].astype(str)
gdf_trees["pitid"] = gdf_trees["pitid"].astype(str)


In [17]:
# keep only trees which have ndvi
gdf_ndvi_trees = gdf_trees.merge(
    df_ndvi,
    left_on="pitid",
    right_on="tree_id",
    how="inner"
)

In [18]:
# merge with watering data
df_water = df_water.reset_index()

gdf_ndvi_trees = gdf_ndvi_trees.merge(
    df_water,
    left_on="pitid",
    right_on="id",
    how="left"
)


In [19]:
# fill missing watering
water_cols = [c for c in gdf_ndvi_trees.columns if c.startswith("water_m")]
gdf_ndvi_trees[water_cols] = gdf_ndvi_trees[water_cols].fillna(0)

In [20]:
# merge with weather data
gdf_ndvi_trees = gdf_ndvi_trees.merge(
    df_weather,
    how="cross"
)

gdf_ndvi_trees.head(5)

,id_x,gisid,pitid,standortnr,kennzeich,namenr,art_dtsch,art_bot,gattung_deutsch,gattung,...,temp_m7,temp_m8,temp_m9,precip_m3,precip_m4,precip_m5,precip_m6,precip_m7,precip_m8,precip_m9
0,strassenbaeume.00008100_000c09d7,00008100_000c09d7,00008100:000c09d7,48,01380,Friedbergstraße,Echter Rotdorn,Crataegus laevigata 'Paul´s Scarlet',WEIßDORN,CRATAEGUS,...,20.251613,21.754839,13.72,2.2,38.0,50.1,35.1,44.9,63.4,50.9
1,strassenbaeume.00008100_000c09e9,00008100_000c09e9,00008100:000c09e9,30,01380,Friedbergstraße,Echter Rotdorn,Crataegus laevigata 'Paul´s Scarlet',WEIßDORN,CRATAEGUS,...,20.251613,21.754839,13.72,2.2,38.0,50.1,35.1,44.9,63.4,50.9
2,strassenbaeume.00008100_000c09ea,00008100_000c09ea,00008100:000c09ea,29,01380,Friedbergstraße,Echter Rotdorn,Crataegus laevigata 'Paul´s Scarlet',WEIßDORN,CRATAEGUS,...,20.251613,21.754839,13.72,2.2,38.0,50.1,35.1,44.9,63.4,50.9
3,strassenbaeume.00008100_000c0a20,00008100_000c0a20,00008100:000c0a20,49,01362,Fredericiastraße,Fächerblattbaum,Ginkgo biloba,GINKGO,GINKGO,...,20.251613,21.754839,13.72,2.2,38.0,50.1,35.1,44.9,63.4,50.9
4,strassenbaeume.00008100_000c0a21,00008100_000c0a21,00008100:000c0a21,48,01362,Fredericiastraße,Fächerblattbaum,Ginkgo biloba,GINKGO,GINKGO,...,20.251613,21.754839,13.72,2.2,38.0,50.1,35.1,44.9,63.4,50.9


In [21]:
gdf_ndvi_trees.columns

Index(['id_x', 'gisid', 'pitid', 'standortnr', 'kennzeich', 'namenr',
       'art_dtsch', 'art_bot', 'gattung_deutsch', 'gattung', 'art_gruppe',
       'strname', 'hausnr', 'zusatz', 'pflanzjahr', 'standalter', 'kronedurch',
       'stammumfg', 'baumhoehe', 'eigentuemer', 'bezirk', 'geometry',
       'tree_id', 'NDVI_m3', 'NDVI_m4', 'NDVI_m5', 'NDVI_m6', 'NDVI_m7',
       'NDVI_m8', 'NDVI_m9', 'has_watering', 'index', 'id_y', 'water_m3',
       'water_m4', 'water_m5', 'water_m6', 'water_m7', 'water_m8', 'water_m9',
       'temp_m3', 'temp_m4', 'temp_m5', 'temp_m6', 'temp_m7', 'temp_m8',
       'temp_m9', 'precip_m3', 'precip_m4', 'precip_m5', 'precip_m6',
       'precip_m7', 'precip_m8', 'precip_m9'],
      dtype='object')

In [ ]:
# select which columns to keep
keep_cols = (
    # identifiers
    ["pitid", "geometry", "bezirk"] +

    # tree characteristics 
    # Tree taxonomy was included at the functional group, genus, and species level to control for physiological differences affecting vegetation indices.
    ["art_gruppe", "gattung", "art_bot", "pflanzjahr",
     "stammumfg", "baumhoehe", "kronedurch"] +

    # NDVI
    [f"NDVI_m{m}" for m in range(3, 10)] +

    # watering
    [f"water_m{m}" for m in range(3, 10)] +

    # weather
    [f"temp_m{m}" for m in range(3, 10)] +
    [f"precip_m{m}" for m in range(3, 10)] +

    # derived
    ["has_watering"]
)

In [28]:
missing = set(keep_cols) - set(gdf_ndvi_trees.columns)
assert not missing, f"Missing expected columns: {missing}"

gdf_clean = gdf_ndvi_trees[keep_cols].copy()
gdf_clean.head(5)

,pitid,geometry,bezirk,art_gruppe,gattung,art_bot,pflanzjahr,stammumfg,baumhoehe,kronedurch,...,temp_m8,temp_m9,precip_m3,precip_m4,precip_m5,precip_m6,precip_m7,precip_m8,precip_m9,has_watering
0,00008100:000c09d7,POINT (13.29195 52.50365),Charlottenburg-Wilmersdorf,Laubbäume,CRATAEGUS,Crataegus laevigata 'Paul´s Scarlet',1991,40.0,7.0,4.0,...,21.754839,13.72,2.2,38.0,50.1,35.1,44.9,63.4,50.9,True
1,00008100:000c09e9,POINT (13.29324 52.50396),Charlottenburg-Wilmersdorf,Laubbäume,CRATAEGUS,Crataegus laevigata 'Paul´s Scarlet',2001,45.0,7.0,4.0,...,21.754839,13.72,2.2,38.0,50.1,35.1,44.9,63.4,50.9,True
2,00008100:000c09ea,POINT (13.29334 52.50397),Charlottenburg-Wilmersdorf,Laubbäume,CRATAEGUS,Crataegus laevigata 'Paul´s Scarlet',1991,60.0,7.0,4.0,...,21.754839,13.72,2.2,38.0,50.1,35.1,44.9,63.4,50.9,True
3,00008100:000c0a20,POINT (13.28022 52.51082),Charlottenburg-Wilmersdorf,Nadelbäume,GINKGO,Ginkgo biloba,1990,52.0,12.0,2.0,...,21.754839,13.72,2.2,38.0,50.1,35.1,44.9,63.4,50.9,True
4,00008100:000c0a21,POINT (13.28009 52.51081),Charlottenburg-Wilmersdorf,Nadelbäume,GINKGO,Ginkgo biloba,1992,50.0,9.0,3.0,...,21.754839,13.72,2.2,38.0,50.1,35.1,44.9,63.4,50.9,True


In [29]:
# save the data set
gdf_clean.to_file(
    "data/timeseries/trees_ndvi_weather_water.gpkg",
    layer="trees",
    driver="GPKG"
)